# fastNLP开发进阶教程


## 组织数据部分
## DataSet & Instance
fastNLP用DataSet和Instance保存和处理数据。每个DataSet表示一个数据集，每个Instance表示一个数据样本。一个DataSet存有多个Instance，每个Instance可以自定义存哪些内容。

In [1]:
# 声明部件
import torch
import fastNLP
from fastNLP import DataSet
from fastNLP import Instance
from fastNLP import Vocabulary
from fastNLP import Trainer
from fastNLP import Tester

### Instance
Instance表示一个样本，由一个或者多个field（域、属性、特征）组成，每个field具有自己的名字以及值
在初始化Instance的时候可以定义它包含的field，使用"field_name=field_value"的写法

In [2]:
# 组织一个Instance，这个Instance由premise、hypothesis、label三个field组成
instance = Instance(premise='an premise example .', hypothesis='an hypothesis example.', label=1)
instance

{'premise': an premise example . type=str,
'hypothesis': an hypothesis example. type=str,
'label': 1 type=int}

In [3]:
data_set = DataSet([instance] * 5)
data_set.append(instance)
data_set[-2: ]

DataSet({'premise': an premise example . type=str,
'hypothesis': an hypothesis example. type=str,
'label': 1 type=int},
{'premise': an premise example . type=str,
'hypothesis': an hypothesis example. type=str,
'label': 1 type=int})

In [4]:
# 如果某一个field的类型与dataset对应的field类型不一样仍可被加入dataset中
instance2 = Instance(premise='the second premise example .', hypothesis='the second hypothesis example.', label='1')
try:
    data_set.append(instance2)
except:
    pass
data_set[-2: ]

DataSet({'premise': an premise example . type=str,
'hypothesis': an hypothesis example. type=str,
'label': 1 type=int},
{'premise': the second premise example . type=str,
'hypothesis': the second hypothesis example. type=str,
'label': 1 type=str})

In [5]:
# 如果某一个field的名字不对，则该instance不能被append到dataset中
instance3 = Instance(premises='the third premise example .', hypothesis='the third hypothesis example.', label=1)
try:
    data_set.append(instance3)
except:
    print('cannot append instance')
    pass
data_set[-2: ]

cannot append instance


DataSet({'premise': an premise example . type=str,
'hypothesis': an hypothesis example. type=str,
'label': 1 type=int},
{'premise': the second premise example . type=str,
'hypothesis': the second hypothesis example. type=str,
'label': 1 type=str})

In [6]:
# 除了文本以外，还可以将tensor作为其中一个field的value
import torch
tensor_ins = Instance(image=torch.randn(5, 5), label=0)
ds = DataSet()
ds.append(tensor_ins)
ds

DataSet({'image': tensor([[ 0.3582, -1.0358,  1.4785, -1.5288, -0.9982],
        [-0.3973, -0.4294,  0.9215, -1.9631, -1.6556],
        [ 0.3313, -1.7714,  0.8729,  0.6976, -1.3172],
        [-0.6403,  0.5023, -0.9919,  1.1178, -0.3710],
        [-0.3692,  1.8631, -1.3646, -0.7290, -1.0774]]) type=torch.Tensor,
'label': 0 type=int})

### DataSet
### 使用现有代码读取并组织DataSet
在DataSet类当中有一些read_* 方法，可以从文件中读取数据并组织DataSet

In [7]:
import fastNLP
from fastNLP import DataSet
from fastNLP import Instance

# 从csv读取数据到DataSet
# 类csv文件，即每一行为一个example的文件，都可以使用这种方法进行数据读取
dataset = DataSet.read_csv('tutorial_sample_dataset.csv', headers=('raw_sentence', 'label'), sep='\t')
# 查看DataSet的大小
len(dataset)

77

In [8]:
# 使用数字索引[k]，获取第k个样本
dataset[0]

{'raw_sentence': A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=str}

In [9]:
# 获取的样本是一个Instance
type(dataset[0])

fastNLP.core.instance.Instance

In [10]:
# 使用数字索引[a: b]，获取第a到第b个样本
dataset[0: 3]

DataSet({'raw_sentence': A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=str},
{'raw_sentence': This quiet , introspective and entertaining independent is worth seeking . type=str,
'label': 4 type=str},
{'raw_sentence': Even fans of Ismail Merchant 's work , I suspect , would have a hard time sitting through this one . type=str,
'label': 1 type=str})

In [11]:
# 索引也可以是负数
dataset[-1]

{'raw_sentence': A film that clearly means to preach exclusively to the converted . type=str,
'label': 2 type=str}

### 自行读取并组织DataSet
以SNLI数据集为例，
SNLI数据集的训练、验证、测试集分别三个文件组成：第一个文件每一行是一句话，代表一个example当中的premise；第二个文件每一行也是一句话，代表一个example当中的hypothesis；第三个文件每一行是一个label

In [12]:
data_path = ['premise', 'hypothesis', 'label']

# 读入文件
with open(data_path[0]) as f:
    premise = f.readlines()

with open(data_path[1]) as f:
    hypothesis = f.readlines()

with open(data_path[2]) as f:
    label = f.readlines()

assert len(premise) == len(hypothesis) and len(hypothesis) == len(label)

# 组织DataSet
data_set = DataSet()
for p, h, l in zip(premise, hypothesis, label):
    p = p.strip()  # 将行末空格去除
    h = h.strip()  # 将行末空格去除
    data_set.append(Instance(premise=p, hypothesis=h, truth=l))

data_set[0]

{'premise': A person on a horse jumps over a broken down airplane . type=str,
'hypothesis': A person is training his horse for a competition . type=str,
'truth': 1
 type=str}

### DataSet的其他操作
在构建完毕DataSet后，仍然可以对DataSet的内容进行操作，函数接口为DataSet.apply()

In [13]:
# 将premise域的所有文本转成小写
data_set.apply(lambda x: x['premise'].lower(), new_field_name='premise')
data_set[-2: ]

DataSet({'premise': a woman is walking across the street eating a banana , while a man is following with his briefcase . type=str,
'hypothesis': An actress and her favorite assistant talk a walk in the city . type=str,
'truth': 1
 type=str},
{'premise': a woman is walking across the street eating a banana , while a man is following with his briefcase . type=str,
'hypothesis': a woman eating a banana crosses a street type=str,
'truth': 0
 type=str})

In [14]:
# label转int
data_set.apply(lambda x: int(x['truth']), new_field_name='truth')
data_set[-2: ]

DataSet({'premise': a woman is walking across the street eating a banana , while a man is following with his briefcase . type=str,
'hypothesis': An actress and her favorite assistant talk a walk in the city . type=str,
'truth': 1 type=int},
{'premise': a woman is walking across the street eating a banana , while a man is following with his briefcase . type=str,
'hypothesis': a woman eating a banana crosses a street type=str,
'truth': 0 type=int})

In [15]:
# 使用空格分割句子
def split_sent(ins):
    return ins['premise'].split()
data_set.apply(split_sent, new_field_name='premise')
data_set.apply(lambda x: x['hypothesis'].split(), new_field_name='hypothesis')
data_set[-2:]

DataSet({'premise': ['a', 'woman', 'is', 'walking', 'across', 'the', 'street', 'eating', 'a', 'banana', ',', 'while', 'a', 'man', 'is', 'following', 'with', 'his', 'briefcase', '.'] type=list,
'hypothesis': ['An', 'actress', 'and', 'her', 'favorite', 'assistant', 'talk', 'a', 'walk', 'in', 'the', 'city', '.'] type=list,
'truth': 1 type=int},
{'premise': ['a', 'woman', 'is', 'walking', 'across', 'the', 'street', 'eating', 'a', 'banana', ',', 'while', 'a', 'man', 'is', 'following', 'with', 'his', 'briefcase', '.'] type=list,
'hypothesis': ['a', 'woman', 'eating', 'a', 'banana', 'crosses', 'a', 'street'] type=list,
'truth': 0 type=int})

In [16]:
# 筛选数据
origin_data_set_len = len(data_set)
data_set.drop(lambda x: len(x['premise']) <= 6)
origin_data_set_len, len(data_set)

(100, 97)

In [17]:
# 增加长度信息
data_set.apply(lambda x: [1] * len(x['premise']), new_field_name='premise_len')
data_set.apply(lambda x: [1] * len(x['hypothesis']), new_field_name='hypothesis_len')
data_set[-1]

{'premise': ['a', 'woman', 'is', 'walking', 'across', 'the', 'street', 'eating', 'a', 'banana', ',', 'while', 'a', 'man', 'is', 'following', 'with', 'his', 'briefcase', '.'] type=list,
'hypothesis': ['a', 'woman', 'eating', 'a', 'banana', 'crosses', 'a', 'street'] type=list,
'truth': 0 type=int,
'premise_len': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] type=list,
'hypothesis_len': [1, 1, 1, 1, 1, 1, 1, 1] type=list}

In [18]:
# 设定特征域、标签域
data_set.rename_field("premise","words1")
data_set.rename_field("premise_len","seq_len1")
data_set.rename_field("hypothesis","words2")
data_set.rename_field("hypothesis_len","seq_len2")
data_set.set_input("words1", "seq_len1", "words2", "seq_len2")
data_set.set_target("truth")

In [19]:
# 重命名field
data_set.rename_field('truth', 'label')
data_set[-1]

{'words1': ['a', 'woman', 'is', 'walking', 'across', 'the', 'street', 'eating', 'a', 'banana', ',', 'while', 'a', 'man', 'is', 'following', 'with', 'his', 'briefcase', '.'] type=list,
'seq_len1': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] type=list,
'words2': ['a', 'woman', 'eating', 'a', 'banana', 'crosses', 'a', 'street'] type=list,
'seq_len2': [1, 1, 1, 1, 1, 1, 1, 1] type=list,
'label': 0 type=int}

In [20]:
# 切分训练、验证集、测试集
train_data, vad_data = data_set.split(0.5)
dev_data, test_data = vad_data.split(0.4)
len(train_data), len(dev_data), len(test_data)

(49, 29, 19)

In [21]:
# 深拷贝一个数据集
import copy
train_data_2, dev_data_2 = copy.deepcopy(train_data), copy.deepcopy(dev_data)
del copy

### DataSet的总结：
将DataSet的ield设置为input和target后，这些field在接下来的代码中将被使用。其中被设置为input的field会被传递给<strong>Model.forward</strong>，这个过程是通过键匹配的方式进行的。举例如下：  
假设DataSet中有'x1', 'x2', 'x3'被设置为了input，而  
&emsp;&emsp;&emsp;(1)函数是Model.forward(self, x1, x3), 那么DataSet中'x1', 'x3'会被传递给forward函数。多余的'x2'会被忽略  
&emsp;&emsp;&emsp;(2)函数是Model.forward(self, x1, x4), 这里多需要了一个'x4', 但是DataSet的input field中没有这个field，会报错。  
&emsp;&emsp;&emsp;(3)函数是Model.forward(self, x1, kwargs), 会把'x1', 'x2', 'x3'都传入。但如果是Model.forward(self, x4, kwargs)就会发生报错，因为没有'x4'。  
&emsp;&emsp;&emsp;(4)对于设置为target的field的名称，我们建议取名为'target'（如果只有一个需要predict的值），但是不强制。如果这个名称不是target，那么在加载loss函数的时候需要手动添加名称转换map

### Vocabulary
fastNLP中的Vocabulary轻松构建词表，并将词转成数字。构建词表有两种方式：根据数据集构建词表；载入现有词表
### 根据数据集构建词表

In [23]:
# 初始化词表，该词表最大的vocab_size为10000，词表中每个词出现的最低频率为2，'<unk>'表示未知词语，'<pad>'表示padding词语
# Vocabulary默认初始化参数为max_size=None, min_freq=None, unknown='<unk>', padding='<pad>'
vocab = Vocabulary(max_size=10000, min_freq=2, unknown='<unk>', padding='<pad>')

# 构建词表
train_data.apply(lambda x: [vocab.add(word) for word in x['words1']])
train_data.apply(lambda x: [vocab.add(word) for word in x['words2']])
vocab.build_vocab()

In [24]:
# 根据词表index句子
train_data.apply(lambda x: [vocab.to_index(word) for word in x['words1']], new_field_name='words1')
train_data.apply(lambda x: [vocab.to_index(word) for word in x['words2']], new_field_name='words2')
dev_data.apply(lambda x: [vocab.to_index(word) for word in x['words1']], new_field_name='words1')
dev_data.apply(lambda x: [vocab.to_index(word) for word in x['words2']], new_field_name='words2')
test_data.apply(lambda x: [vocab.to_index(word) for word in x['words1']], new_field_name='words1')
test_data.apply(lambda x: [vocab.to_index(word) for word in x['words2']], new_field_name='words2')
train_data[-1], dev_data[-1], test_data[-1]

({'words1': [2, 9, 4, 2, 75, 85, 7, 86, 76, 77, 87, 88, 89, 2, 90, 3] type=list,
 'seq_len1': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] type=list,
 'words2': [18, 9, 10, 1, 3] type=list,
 'seq_len2': [1, 1, 1, 1, 1] type=list,
 'label': 1 type=int},
 {'words1': [22, 32, 5, 110, 81, 111, 112, 5, 82, 3] type=list,
 'seq_len1': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] type=list,
 'words2': [64, 32, 82, 133, 84, 3] type=list,
 'seq_len2': [1, 1, 1, 1, 1, 1] type=list,
 'label': 0 type=int},
 {'words1': [2, 9, 97, 1, 20, 7, 54, 5, 1, 1, 70, 2, 11, 110, 2, 62, 3] type=list,
 'seq_len1': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] type=list,
 'words2': [23, 1, 58, 10, 12, 1, 70, 133, 84, 3] type=list,
 'seq_len2': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] type=list,
 'label': 1 type=int})

### 载入现有词表
以BERT pretrained model为例，词表由一个vocab.txt文件来保存
用以下方法可以载入现有词表，并保证词表顺序不变

In [25]:
# 读入vocab文件
with open('vocab.txt') as f:
    lines = f.readlines()
vocabs = []
for line in lines:
    vocabs.append(line.strip())

# 实例化Vocabulary
vocab_bert = Vocabulary(unknown=None, padding=None)
# 将vocabs列表加入Vocabulary
vocab_bert.add_word_lst(vocabs)
# 构建词表
vocab_bert.build_vocab()
# 更新unknown与padding的token文本
vocab_bert.unknown = '[UNK]'
vocab_bert.padding = '[PAD]'

In [26]:
# 根据词表index句子
train_data_2.apply(lambda x: [vocab_bert.to_index(word) for word in x['words1']], new_field_name='words1')
train_data_2.apply(lambda x: [vocab_bert.to_index(word) for word in x['words2']], new_field_name='words2')
dev_data_2.apply(lambda x: [vocab_bert.to_index(word) for word in x['words1']], new_field_name='words1')
dev_data_2.apply(lambda x: [vocab_bert.to_index(word) for word in x['words2']], new_field_name='words2')
train_data_2[-1], dev_data_2[-1]

({'words1': [1037, 2450, 1999, 1037, 2665, 6598, 1998, 7415, 2058, 2014, 2132, 2559, 2875, 1037, 3028, 1012] type=list,
 'seq_len1': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1] type=list,
 'words2': [100, 2450, 2003, 3147, 1012] type=list,
 'seq_len2': [1, 1, 1, 1, 1] type=list,
 'label': 1 type=int},
 {'words1': [2048, 2308, 1010, 3173, 2833, 100, 16143, 1010, 8549, 1012] type=list,
 'seq_len1': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] type=list,
 'words2': [100, 2308, 8549, 2169, 2060, 1012] type=list,
 'seq_len2': [1, 1, 1, 1, 1, 1] type=list,
 'label': 0 type=int})

## 模型部分
## Model
模型部分fastNLP提供两种使用方式：调用fastNLP现有模型；开发者自行搭建模型
### 调用fastNLP现有模型

In [27]:
# step 1：加载模型参数（非必选）
from fastNLP.io.config_io import ConfigSection, ConfigLoader
args = ConfigSection()
ConfigLoader().load_config("./data/config", {"esim_model": args})
args["vocab_size"] = len(vocab)
args.data

{'embed_dim': 300,
 'hidden_size': 300,
 'batch_first': True,
 'dropout': 0.3,
 'num_classes': 3,
 'gpu': True,
 'batch_size': 32,
 'vocab_size': 143}

In [28]:
# step 2：加载ESIM模型
from fastNLP.models import ESIM
model = ESIM(args["vocab_size"], args["embed_dim"], args["hidden_size"], args["dropout"], args["num_classes"])
model

ESIM(
  (drop): Dropout(p=0.3)
  (embedding): Embedding(
    143, 300
    (dropout): Dropout(p=0.3)
  )
  (embedding_layer): Linear(in_features=300, out_features=300, bias=True)
  (encoder): LSTM(
    (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (bi_attention): BiAttention()
  (mean_pooling): MeanPoolWithMask()
  (max_pooling): MaxPoolWithMask()
  (inference_layer): Linear(in_features=1200, out_features=300, bias=True)
  (decoder): LSTM(
    (lstm): LSTM(300, 300, batch_first=True, bidirectional=True)
  )
  (output): MLP(
    (hiddens): ModuleList(
      (0): Linear(in_features=1200, out_features=300, bias=True)
    )
    (output): Linear(in_features=300, out_features=3, bias=True)
    (dropout): Dropout(p=0.3)
  )
)

这是上述模型的forward方法。如果你不知道什么是forward方法，请参考我们的PyTorch教程。

注意两点：
1. forward参数名字叫**word_seq**，请记住。
2. forward的返回值是一个**dict**，其中有个key的名字叫**pred**。

```Python
    def forward(self, word_seq):
        """

        :param word_seq: torch.LongTensor, [batch_size, seq_len]
        :return output: dict of torch.LongTensor, [batch_size, num_classes]
        """
        x = self.embed(word_seq)  # [N,L] -> [N,L,C]
        x = self.conv_pool(x)  # [N,L,C] -> [N,C]
        x = self.dropout(x)
        x = self.fc(x)  # [N,C] -> [N, N_class]
        return {'pred': x}
```

### 自行搭载模型
自行搭载的模型必须是nn.Module的子类，  
(1)必须实现forward方法，并且forward方法不能出现**\*arg**这种参数，例如
```Python
    def forword(self, word_seq, *args): # 这是不允许的
        xxx
```
forward函数的返回值必须是一个**dict**。  
dict当中模型预测的值所对应的key建议用**'pred'**，这里不做强制限制，但是如果不是pred的话，在加载loss函数的时候需要手动添加名称转换map  
(2)如果实现了predict方法，在做evaluation的时候将调用predict方法而不是forward。如果没有predict方法，则在evaluation时调用forward方法。predict方法也不能使用\*args这种参数形式，同时结果也必须返回一个dict，同样推荐key为'pred'。  
(3)forward函数可以计算loss并返回结果，在dict中的key为'loss'，如：  
```Python
    def forword(self, word_seq, *args): 
        xxx
        return {'pred': pred, 'loss': loss}
```
当loss函数没有在trainer里被定义的时候，trainer将会根据forward函数返回的dict中key为'loss'的值来进行反向传播，具体见loss部分

## 训练测试部分
## Loss
### 键映射
根据上文DataSet与Model部分可以知道，fastNLP并不限制Model.forward()的返回值，也不限制DataSet中target field的key。因此在计算loss的时候，需要通过键映射的方式来完成取值。  
这里以CrossEntropyLoss为例，我们的交叉熵函数部分如下：
```Python
    def get_loss(self, pred, target):
        return F.cross_entropy(input=pred, target=target,
                               ignore_index=self.padding_idx)
```
这里接收的两个参数名字分别为pred和target，其中pred是从model的forward函数返回值中取得，target是从DataSet的is_target的field当中取得。在没有设置键映射的基础上，pred从model的forward函数返回的dict中取'pred'键得到；target从DataSet的'target'field中得到。
。

### 修改键映射
在初始化CrossEntropyLoss的时候，可以传入两个参数(pred=None, target=None), 这两个参数接受的类型是str，假设(pred='output', target='label')，那么CrossEntropyLoss会使用'output'这个key在forward的output与batch_y中寻找值;'label'也是在forward的output与d ataset的is target field中寻找值。注意这里pred或target的来源并不一定非要来自于model.forward与dataset的is target field，也可以只来自于forward的结果

### 创建一个自己的loss
&emsp;&emsp;&emsp;&emsp;(1)采用fastNLP.LossInForward，在model.forward()的结果中包含一个'loss'的key，具体见**自行搭载模型**部分  
&emsp;&emsp;&emsp;&emsp;(2)自己定义一个继承于fastNLP.core.loss.LossBase的class。重写get_loss方法。  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;(2.1)在初始化自己的loss class的时候，需要初始化需要映射的值  
&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;(2.2)在get_loss函数中，参数的名字需要与初始化时的映射是一致的  
以L1Loss为例子：
```Python
class L1Loss(LossBase):
    def __init__(self, pred=None, target=None):
        super(L1Loss, self).__init__()
        """
        这里传入_init_param_map以使得pred和target被正确注册，但这一步不是必须的, 建议调用。传入_init_param_map的是用于
        "键映射"的键值对。假设初始化__init__(pred=None, target=None, threshold=0.1)中threshold是用于控制loss计算的，
        则\不要将threshold传入_init_param_map.
        """
        self._init_param_map(pred=pred, target=target)

    def get_loss(self, pred, target):
        # 这里'pred', 'target'必须和初始化的映射是一致的。
        return F.l1_loss(input=pred, target=target)
```


## Trainer
trainer的作用是训练模型，是一个对训练过程的封装。trainer当中比较重要的函数是trainer.train()，train函数的主要步骤包括：  
(1)创建batch  
```Python
batch = Batch(dataset, batch_size, sampler=sampler)
```
(2)for batch_x, batch_y in batch: (batch_x, batch_y的内容分别为dataset中is input和is target的部分，这两个dict的key就是DataSet中的key，value会根据情况做好padding及tensor)  
&emsp;&emsp;(2.1)将batch_x, batch_y中的tensor移动到model所在的device  
&emsp;&emsp;(2.2)根据model.forward的参数列表，从batch_x中取出需要传递给forward的数据  
&emsp;&emsp;(2.3)获取model.forward返回的dict，并与batch_y一起传递给loss函数，求得loss  
&emsp;&emsp;(2.4)对loss进行反向梯度传播并更新参数  
(3)如果有验证集，则进行验证  
(4)如果验证集的结果是当前最佳结果，则保存模型

除了以上的内容， Trainer中还提供了"预跑"的功能。该功能通过check_code_level管理，如果check_code_level为-1，则不进行"预跑"。 check_code_level=0，1，2代表不同的提醒级别。目前不同提醒级别对应的是对DataSet中设置为input或target但又没有使用的field的提醒级别。 0是忽略(默认)；1是会warning发生了未使用field的情况；2是出现了unused会直接报错并退出运行 "预跑"的主要目的有两个:   
(1)防止train完了之后进行evaluation的时候出现错误。之前的train就白费了  
(2)由于存在"键映射"，直接运行导致的报错可能不太容易debug，通过"预跑"过程的报错会有一些debug提示 "预跑"会进行以下的操作：  
&emsp;&emsp;(i) 使用很小的batch_size, 检查batch_x中是否包含Model.forward所需要的参数。只会运行两个循环。  
&emsp;&emsp;(ii)将Model.foward的输出pred_dict与batch_y输入到loss中，并尝试backward。不会更新参数，而且grad会被清零。如果传入了dev_data，还将进行metric的测试  
&emsp;&emsp;(iii)创建Tester，并传入少量数据，检测是否可以正常运行  
"预跑"操作是在Trainer初始化的时候执行的。正常情况下，应该不需要改动"预跑"的代码。但如果遇到bug或者有什么好的建议，欢迎在开发群或者github提交issue。

In [29]:
from fastNLP import CrossEntropyLoss
from fastNLP import Adam
from fastNLP import AccuracyMetric
trainer = Trainer(
    train_data=train_data,
    model=model,
    loss=CrossEntropyLoss(pred='pred', target='label'), # 模型预测值通过'pred'来取得，目标值（ground truth）由'label'取得
    metrics=AccuracyMetric(target='label'), # 目标值（ground truth）由'label'取得
    n_epochs=5,
    batch_size=16,
    print_every=-1,
    validate_every=-1,
    dev_data=dev_data,
    optimizer=Adam(lr=1e-3, weight_decay=0),
    check_code_level=-1,
    metric_key='acc',
    use_tqdm=False,
)
trainer.train()

training epochs started 2019-05-14-19-49-25


AssertionError: seq_len can only have one dimension, got False.

### Tester
tester的作用是训练模型，是一个对训练过程的封装。tester当中比较重要的函数是tester.test()，test函数的主要步骤包括：
(1)创建batch
```Python
batch = Batch(dataset, batch_size, sampler=sampler)
```
(2)for batch_x, batch_y in batch: (batch_x, batch_y的内容分别为dataset中is input和is target的部分，这两个dict的key就是DataSet中的key，value会根据情况做好padding及tensor)  
&emsp;&emsp;(2.1)同步数据与model将batch_x, batch_y中的tensor移动到model所在的device  
&emsp;&emsp;(2.2)根据predict_func的参数列表，从batch_x中取出需要传递给predict_func的数据，得到结果pred_dict  
&emsp;&emsp;(2.3)调用metric(pred_dict, batch_y)  
&emsp;&emsp;(2.4)当所有batch都运行完毕，会调用metric的get_metric方法，并且以返回的值作为evaluation的结果  

In [30]:
tester = Tester(
    data=test_data,
    model=model,
    metrics=AccuracyMetric(target='label'),
    batch_size=args["batch_size"],
)
tester.test()

[tester] 
AccuracyMetric: acc=0.368421


{'AccuracyMetric': {'acc': 0.368421}}